# Notebook 699: Tarea calificada 2, INAR 23-24

## Generación de texto seq2seq model
## A partir de textos de parlamentarios españoles (anteriores a 2022)

## Nota importante

Esta tarea en su versión 2023-24 surge del excelente trabajo de varios compañeros del curso 2022-23, que aunque yo proporcioné un dataset de textos a partir de las intervenciones de parlamentarios (los líderes de varios partidos en 2021-22, alguno de los cuales ya no está en la política española), hicieron un extraordinario "escrapeo" de la web del Congreso de los Diputados y enriquecieron de forma notable el dataset. Este es el que propongo para esta tarea.

Debo decir que si hay un texto (o lenguaje natural) libre de derechos y especialmente actual, son las intervenciones (estrictamente **públicas**) de los representantes elegidos en elecciones, y que el Congreso debería facilitar, no ya para su uso en estas tareas, sino para cualquier estudioso del español, o de la política, o de la psicología de los políticos.

Por supuesto, esto son opiniones estrictamete mías, en el momento concreto en que las escribo, y sencillamente quiero hacer homenaje a los que colaboraron tanto con este trabajo que espero encontréis interesante.

## ¿De qué trata esta tarea?

Pues ni más ni menos que de generar texto en español a partir de texto de parlamentarios, basado en el tutorial que hemos seguido en clase:

https://www.tensorflow.org/text/tutorials/text_generation?hl=es-419

Para facilitar la tarea se propone un pre-proceso (basado en la tarea 2021-22), y la tarea se concreta en el modelo para generar texto y en las pruebas de la calidad del texto generado.


## Calificación

Está explicada en la entrada correspondiente de Blackboard. Básicamente, hay un mínimo que consiste en proponer tres modelos de red recurrente, uno para cada parlamentario, entrenarlos, y **evaluarlos** generando texto y comentando su calidad.

Para llegar a la máxima nota, propongo poner a dialogar los tres modelos.

Pero por supuesto, valoraré el trabajo de construcción del modelo. Para esta tarea no hay una "medida" como la accuracy en la tarea 1. Será relativamente subjetiva. Por eso parece aconsejable comenzar con modelos pequeños o con pocas etapas e ir refinando.

## Setup

Para facilitar la tarea propongo unas cuantas casillas para cargar en memoria los textos, tres .txt que están incluidos en un .zip.

## Nota importante

La codificación (juego de caracteres) es UTF-8 y creo que debe seguir siendo así. *NO* abráis los .txt con el Notepad de Windows, sino con el Notepad+++ que os permitiría cambiarlo o devolverlo a UTF-8 (o Unicode si queréis).

A pesar que la salida por pantalla (en mi sistema, un Linux) de caracteres ñ y acentuados parece que está mal, luego la generación de texto (insisto, lo he comprobado en mi sistema) es correcta en español.


### Import TensorFlow and other libraries

In [34]:
import tensorflow as tf
import numpy as np
import os


## Lectura de ficheros de datos

In [35]:
datos_abascal   = "intervencionesAbascal.txt"
datos_sanchez   = "intervencionesSanchez.txt"
datos_casado    = "intervencionesCasado.txt"

### Leer los ficheros de datos

Primero, abrimos el texto de Santiago Abascal, que es el más corto, y lo leemos:

In [36]:
# Read, then decode for py2 compat.
text = open(datos_abascal, 'rb').read().decode(encoding='utf-8')

# length of text is the number of characters in it
print(f'Texto de Santiago Abascal: {len(text)} carácteres')

Texto de Santiago Abascal: 22573 carácteres


In [37]:
# Take a look at the first 250 characters in text
print(text[:250])

Señor Sánchez, ¿cómo se atreve usted a hablarme de monólogos si siempre trae las respuestas escritas, si usted nunca contesta a mis preguntas? Conteste por lo menos hoy. ¿Qué va a hacer usted para impedir que VOX siga cruzando las líneas que dice ust


In [38]:
# The unique characters in the file
vocab = sorted(set(text))
print(f'{len(vocab)} unique characters')

81 unique characters


Vamos a ver cual de los tres textos tiene el mayor vocabulario, para usar el mismo en los tres modelos:

In [39]:
vocab_mayor = vocab

textos = []

for texto in [datos_abascal, datos_sanchez, datos_casado]:
    text = open(texto, 'rb').read().decode(encoding='utf-8')
    vocab = sorted(set(text))
    print(f'{len(text)} carácteres, {len(vocab)} únicos en {texto}')
    
    if len(vocab) > len(vocab_mayor):
        vocab_mayor = vocab
    
    textos.append(text)
        
vocab = sorted(set(textos[0] + textos[1] + textos[2]))
vocab_size = len(vocab)

print(f'{vocab_size} únicos en los tres textos')

22573 carácteres, 81 únicos en intervencionesAbascal.txt
239623 carácteres, 104 únicos en intervencionesSanchez.txt
105940 carácteres, 92 únicos en intervencionesCasado.txt
108 únicos en los tres textos


## Procesar el texto

### Vamos a vectorizar el texto

Como las redes neuronales no entienden carácteres sino números, vamos a vectorizar el texto. Para ello, vamos a crear dos *"tablas de traducción"*, uno para pasar de carácter a número y otro para pasar de número a carácter.

In [40]:
# Creamos un diccionario para asignar cada caracter a un entero
char2idx = {u:i for i, u in enumerate(vocab)}

# Luego hacemos una lista con los carácteres ordenados por su entero
idx2char = np.array(vocab)

# Vamos a ver que pinta tiene nuestro diccionario
for char,_ in zip(char2idx, range(20)):
    print(f'{repr(char)}: {char2idx[char]}')

'\x07': 0
'\n': 1
' ': 2
'!': 3
'%': 4
'&': 5
'(': 6
')': 7
',': 8
'-': 9
'.': 10
'0': 11
'1': 12
'2': 13
'3': 14
'4': 15
'5': 16
'6': 17
'7': 18
'8': 19


Ahora ya podemos vectorizar el texto

In [41]:
# Ahora podemos convertir todo el texto a enteros
text_as_int = np.array([char2idx[c] for c in text])

# Vamos a ver como queda el texto en enteros
print(f'{repr(text[:13])} ---- carácteres mapeados a int ----> {text_as_int[:13]}')

'Señor Sánchez' ---- carácteres mapeados a int ----> [42 56 93 66 69  2 42 90 65 54 59 56 77]


# Fases propuestas para la elaboración del modelo

### 1. Crear los training examples y los targets

Ahora vamos a divir nuestro texto en secuencia de carácteres. Cada secuencia tendrá `seq_length` carácteres de nuestro texto.
Para cada secuencia de entrada, los targets correspondientes contienen la misma longitud de texto, excepto desplazada un carácter a la derecha.
Por eso dividimos el texto en secuencias de `seq_length+1`. Por ejemplo, digamos que `seq_length` es 4 y nuestro texto es "Hola". La secuencia de entrada sería "Hol" y la secuencia de salida "ola".

Para hacer esto, primero usamos la función `tf.data.Dataset.from_tensor_slices` para convertir el vector de texto en una secuencia de índices de caracteres.



In [42]:
# Creamos un dataset de tensorflow con los enteros
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

# Ahora vamos a dividir el texto en secuencias de 100 carácteres
SEQ_LENGTH = 100
sequences = char_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

# Vamos a ver como son estas secuencias
print("Secuencias de 100 carácteres:")
for item in sequences.take(5):
    print(repr(''.join(idx2char[item.numpy()])))

Secuencias de 100 carácteres:
'Señor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima propon'
'en las mismas recetas fracasadas que nos llevaron a la peor crisis económica de nuestra historia: más'
' despilfarro, más déficit y más impuestos. Pero el Partido Popular es un partido de Estado y también '
'de Gobierno, aunque estemos temporalmente en la oposición. Por eso el lunes le ofrecí pactar los Pres'
'upuestos Generales si rompe con los independentistas, una oferta, por cierto, a la que usted no ha co'


El conjunto de datos de entrenamiento contiene tanto los datos de entrada (desde la posición 0 a la 99) como los de salida (desde la posición 1 a la 100). Por lo que necesitamos mapear el input y el target para crear el dataset.

In [43]:
def split_input_target(sequence):
    input_text = sequence[:-1]
    target_text = sequence[1:]
    return input_text, target_text

# Ahora vamos a aplicar la función anterior a todas las secuencias
dataset = sequences.map(split_input_target)

# Vamos a ver como son las secuencias de entrada y salida
for input_example, target_example in  dataset.take(1):
    print ('Input: ', repr(''.join(idx2char[input_example.numpy()])))
    print ('Target: ', repr(''.join(idx2char[target_example.numpy()])))

Input:  'Señor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima propo'
Target:  'eñor Sánchez, sus recetas económicas son tan creíbles como sus promesas electorales, y encima propon'


### 2. Crear los training batches

Ahora ya podemos mezclar los datos y empaquetarlos en batches de 64 secuencias.

In [44]:
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)

dataset

<_BatchDataset element_spec=(TensorSpec(shape=(64, 100), dtype=tf.int64, name=None), TensorSpec(shape=(64, 100), dtype=tf.int64, name=None))>

## 3. Crear el modelo

Puedes usar cualquiera de los modelos (RNN, LSTM, GRU) que hemos visto en clase. Por supuesto, del tamaño del modelo (capas, neuronas en cada capa) así como de las épocas (más adelante) dependerá el tiempo de proceso en el .fit

Para el modelo de Abascal vamos a usar una RNN que contenga solo una capa LSTM. En concreto, definiremos una red neuronal de solo 3 capas:

- Capa de entrada: una capa de tipo Embedding, que convierte los índices de los caracteres en vectores embedding de tamaño embedding_dim. En las opciones de la capa especificaremos el tamaño de nuestro vocabulario `(vocab_size)` y el tamaño de los vectores embedding `(embedding_dim)`. También indicaremos el tamaño del batch que vamos a usar `(batch_size)`.

- Capa LSTM: una capa LSTM con `units=2048`, que es el número de neuronas recurrentes de la capa. También indicaremos con return_sequences=True que queremos predecir el carácter siguiente a todos los carácteres de entrada y no solo al último carácter. El argumento `stateful=True` explica el uso de las capacidades de memoria de la red entre batches: Si está en False, por cada nuevo batch se inicializan las memory cells (la parte de la red neuronal que preserva el estado de la red a través del tiempo), pero si está en True, por cada nuevo batch se mantienen las memory cells con las actualizaciones hechas durante la ejecución del batch anterior. El último argumento, `recurrent_initializer='glorot_uniform'`, es un que indica como se inicializan los pesos de las matrices internas de la capa LSTM. En estos casosm la distribución más común es la `glorot_uniform`.

- Capa de salida: una capa Dense con `vocab_size` neuronas. Esta capa nos dará como salida un vector de tamaño `vocab_size` con las probabilidades de que el siguiente carácter sea cada uno de los carácteres del vocabulario.

In [45]:
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        Embedding(vocab_size, embedding_dim,
                  batch_input_shape=[batch_size, None]),
        LSTM(rnn_units, return_sequences=True,
             recurrent_initializer='glorot_uniform',
             stateful=True),
        Dense(vocab_size)
    ])
    
    return model

embedding_dim = 512
rnn_units = 2048

model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.summary()
 

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_3 (Embedding)     (64, None, 512)           55296     
                                                                 
 lstm_2 (LSTM)               (64, None, 2048)          20979712  
                                                                 
 dense_3 (Dense)             (64, None, 108)           221292    
                                                                 
Total params: 21256300 (81.09 MB)
Trainable params: 21256300 (81.09 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


Definimos nuestra función de pérdida y el optimizador que vamos a usar para entrenar el modelo. En este caso, usaremos la función de pérdida `sparse_categorical_crossentropy` y el optimizador `Adam` con sus argumentos por defecto. Con esto ya podemos compilar el modelo.

In [46]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

In [47]:
model.compile(optimizer='adam', loss=loss)

Podemos usar la técnica de los checkpoints para no perder el progreso del entrenamiento si tenemos un fallo en el sistema. El único problema es que los checkpoints pueden llegar a ocupar mucho espacio muy rápidamente, por lo que es recomendable borrarlos después de entrenarlos y en su lugar guardar el modelo ya terminado.

In [48]:
checkpoint_dir = './training_checkpoints'
checkpoint_prefix = os.path.join(checkpoint_dir, "ckpt_{epoch}")

checkpoint_callback=tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_prefix,
    save_weights_only=True)

## 4. Summary y fit del modelo



In [50]:
EPOCHS = 200

history = model.fit(dataset, epochs=EPOCHS, callbacks=[checkpoint_callback])

# Guardamos el modelo y los pesos
model.save_weights('./modelos/abascal_weights.keras')

Epoch 1/200


16/16 [==============================] - 3s 144ms/step - loss: 2.4234
Epoch 2/200
16/16 [==============================] - 2s 125ms/step - loss: 2.3454
Epoch 3/200
16/16 [==============================] - 2s 124ms/step - loss: 2.2828
Epoch 4/200
16/16 [==============================] - 2s 127ms/step - loss: 2.2257
Epoch 5/200
16/16 [==============================] - 2s 125ms/step - loss: 2.1726
Epoch 6/200
16/16 [==============================] - 2s 125ms/step - loss: 2.1147
Epoch 7/200
16/16 [==============================] - 2s 122ms/step - loss: 2.0630
Epoch 8/200
16/16 [==============================] - 2s 122ms/step - loss: 2.0071
Epoch 9/200
16/16 [==============================] - 2s 126ms/step - loss: 1.9552
Epoch 10/200
16/16 [==============================] - 2s 121ms/step - loss: 1.9094
Epoch 11/200
16/16 [==============================] - 2s 123ms/step - loss: 1.8618
Epoch 12/200
16/16 [==============================] - 2s 126ms/step - loss: 1.8149
Epoch 13/200
16/16 [=====

Ahora que hemos terminado de entranar el modelo ya no necesitamos los chekpoints, por lo que podemos borrarlos.

In [51]:
# Borramos los checkpoints para no ocupar espacio

import os, shutil
folder = './training_checkpoints/'

for filename in os.listdir(folder):
    file_path = os.path.join(folder, filename)
    try:
        if os.path.isfile(file_path) or os.path.islink(file_path):
            os.unlink(file_path)
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
    except Exception as e:
        print('Failed to delete %s. Reason: %s' % (file_path, e))

## 5. Genera texto y evalúa su calidad

Para generar texto a partir del modelo, ahora necesitamos un `batch_size` de 1, por lo que tenemos que rehacer el modelo y cargar los pesos de nuestro modelo entrenado.

In [52]:
model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('./modelos/abascal_weights.keras')
model.build(tf.TensorShape([1, None]))

model.save('./modelos/abascal.keras')

Vamos a crear una función que con un texto de entrada nos genere texto. La variable `num_generate` indica cuantos carácteres se generarán y la variable `temperature` indica cuanto varía de los texto originales (con temperaturas altas el modelo será más creativo, pero a costa de cometer más errores). La temperatura está comprendida entre 0 y 1.

In [53]:
def generate_text(model, start_string):
    
    num_generate = 500
    input_eval = [char2idx[s] for s in start_string]
    
    input_eval = tf.expand_dims(input_eval, 0)
    text_generated = []
    temperature = 0.5
    
    model.reset_states()
    
    # Bucle para generar los carácteres
    for i in range(num_generate):
        predictions = model(input_eval)
        predictions = tf.squeeze(predictions, 0) # Reducir la dimensión del batch ya que el modelo está entrenado con batch_size=64 y ahora estamos generando con batch_size=1
        
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()
        
        input_eval = tf.expand_dims([predicted_id], 0)
        
        text_generated.append(idx2char[predicted_id])
        
    return (''.join(text_generated))

In [54]:
print(generate_text(model, start_string=u"La izquierda"))


 y los nacionalistas, y eso que ha hecho trampas con todos los instrumentos del Estado: con el artículo 155? Si no lo hace usted respetar, señor Sánchez. Ha elegido a los radicales, aquellos que van contra la tradición europea, la para proteger a los españoles y deja de insultar a la oposición? Haga algo, señor Sánchez, que para gestionar el mando único y la limitación de movimientos sin tener que recurrir a la legislación básica en vigor, como ya han hecho los países de nuestro entorno. Tal y c


Vemos que enlaza bien la cadena de entrada para que tenga sentido y parece que más o menos tiene sentido lo que dice, pero no es muy coherente. También usa mal los signos de interrogación y algunos artículos. Aun así no es de extrañar, considerando que el dataset de Abascal es el más pequeño de los tres.

## 6. Trabajo adicional

Por ejemplo, poner en cadena los tres modelos para que "dialoguen" entre sí

### Modelo de Pedro Sánchez

El modelo de Pedro Sánchez será similar al de Abascal, una RNN con una capa LSTM.

Creamos el dataset de entrenamiento de Pedro Sánchez

In [55]:
# Abrir el texto
text = open(datos_sanchez, 'rb').read().decode(encoding='utf-8')

# Vectorizar el texto
text_as_int = np.array([char2idx[c] for c in text])

# Secuencias de 100 carácteres
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
SEQ_LENGTH = 100
sequences = char_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

# Dataset de entrenamiento
dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True)
)

Ahora creamos el modelo de Pedro Sánchez

In [56]:
model = build_model(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.compile(optimizer='adam', loss=loss)

Entrenamos el modelo de Pedro Sánchez

In [23]:
EPOCHS = 200

history = model.fit(dataset, epochs=EPOCHS)

# Guardamos el modelo y los pesos
model.save_weights('./modelos/sanchez_weights.keras')

Epoch 1/200
37/37 [==============================] - 5s 113ms/step - loss: 3.3781
Epoch 2/200
37/37 [==============================] - 4s 108ms/step - loss: 2.5602
Epoch 3/200
37/37 [==============================] - 4s 108ms/step - loss: 2.2155
Epoch 4/200
37/37 [==============================] - 4s 108ms/step - loss: 2.0314
Epoch 5/200
37/37 [==============================] - 4s 108ms/step - loss: 1.8559
Epoch 6/200
37/37 [==============================] - 4s 108ms/step - loss: 1.7195
Epoch 7/200
37/37 [==============================] - 4s 108ms/step - loss: 1.5472
Epoch 8/200
37/37 [==============================] - 4s 109ms/step - loss: 1.4205
Epoch 9/200
37/37 [==============================] - 4s 109ms/step - loss: 1.3130
Epoch 10/200
37/37 [==============================] - 4s 109ms/step - loss: 1.2266
Epoch 11/200
37/37 [==============================] - 4s 109ms/step - loss: 1.1507
Epoch 12/200
37/37 [==============================] - 4s 109ms/step - loss: 1.0821
Epoch 13/200


Creamos el modelo para generar texto de Pedro Sánchez

In [57]:
vocab_size = len(vocab)

model = build_model(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('./modelos/sanchez_weights.keras')
model.build(tf.TensorShape([1, None]))

model.save('./modelos/sanchez.keras')

Comprobamos el texto generado por Pedro Sánchez

In [61]:
print(generate_text(model, start_string=u"Los impuestos del"))

 Partido Popular resultar el conjunto de instituciones por aumentar el número de test, que se está elevando. España es uno de los principales valores de nuestra Carta Magna la igualdad, que ustedes atacan cuando el Gobierno de España defenden un objetivo marcado por la Unión Europea de que tiene que haber proyectos incluso transnacionales, sino la de hace ochenta años. La nuestra este Gobierno y, por tanto, no vamos a dejar a nadie quedel 14 de marzo, en Bruselas; del Consejo Europeo, celebrado 


Vemos que ahora el texto generado tiene menos errores gramaticales y está mejor cohesionado, pero sigue sin tener mucho sentido. Vamos a intentar modificar la estructura del modelo para ver si conseguimos mejores resultados. 

### Nuevo modelo de Pedro Sánchez

In [ ]:
from keras.layers import Dropout

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
    model = Sequential([
        Embedding(vocab_size, embedding_dim,
                  batch_input_shape=[batch_size, None]),
        LSTM(rnn_units, return_sequences=True,
             recurrent_initializer='glorot_uniform',
             stateful=True),
        Dropout(0.2),
        LSTM(rnn_units, return_sequences=True,
             recurrent_initializer='glorot_uniform',
             stateful=True),
        Dense(vocab_size)
    ])
    
    return model

embedding_dim = 512
rnn_units = 2048

### Modelo de Pablo Casado (GRU)

Esta vez en vez de usar una LSTM como en los anteriores modelos, vamos a usar una RNN con capa GRU. La GRU es una versión simplificada de la LSTM, que tiene menos parámetros y por lo tanto es más rápida de entrenar. La GRU tiene dos puertas (gates) en vez de tres como la LSTM. 

Que tenga más parámetros le puede dar más capacidad de aprendizaje, pero también puede hacer que el modelo tarde más en entrenar y que sea más propenso al overfitting. Por lo tanto, es posible que la GRU nos venga bien, ya que nuestro dataset es pequeño.

Creamos el dataset de entrenamiento de Pablo Casado

In [26]:
# Abrir el texto
text = open(datos_casado, 'rb').read().decode(encoding='utf-8')

# Vectorizar el texto
text_as_int = np.array([char2idx[c] for c in text])

# Secuencias de 100 carácteres
char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)
SEQ_LENGTH = 100
sequences = char_dataset.batch(SEQ_LENGTH + 1, drop_remainder=True)

# Dataset de entrenamiento
dataset = sequences.map(split_input_target)
BATCH_SIZE = 64
BUFFER_SIZE = 10000

dataset = (
    dataset
    .shuffle(BUFFER_SIZE)
    .batch(BATCH_SIZE, drop_remainder=True))

Creamos el modelo de Pablo Casado, con la capa de GRU en vez de LSTM.

In [15]:
def build_model_gru(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    tf.keras.layers.Embedding(vocab_size,embedding_dim,
    batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units,return_sequences=True,
    stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)]
  )
    
  return model

In [28]:
model = build_model_gru(
    vocab_size=len(vocab),
    embedding_dim=embedding_dim,
    rnn_units=rnn_units,
    batch_size=BATCH_SIZE)

model.compile(optimizer='adam', loss=loss)

Entrenamos el modelo de Pablo Casado

In [29]:
EPOCHS = 200
history = model.fit(dataset, epochs=EPOCHS)

# Guardamos el modelo y los pesos
model.save_weights('./modelos/casado_weights.keras')

Epoch 1/200
16/16 [==============================] - 3s 96ms/step - loss: 5.0457
Epoch 2/200
16/16 [==============================] - 1s 85ms/step - loss: 3.1420
Epoch 3/200
16/16 [==============================] - 1s 86ms/step - loss: 2.7586
Epoch 4/200
16/16 [==============================] - 1s 84ms/step - loss: 2.5321
Epoch 5/200
16/16 [==============================] - 1s 87ms/step - loss: 2.4528
Epoch 6/200
16/16 [==============================] - 1s 84ms/step - loss: 2.4048
Epoch 7/200
16/16 [==============================] - 1s 84ms/step - loss: 2.3686
Epoch 8/200
16/16 [==============================] - 1s 84ms/step - loss: 2.3322
Epoch 9/200
16/16 [==============================] - 1s 86ms/step - loss: 2.2943
Epoch 10/200
16/16 [==============================] - 1s 84ms/step - loss: 2.2577
Epoch 11/200
16/16 [==============================] - 1s 84ms/step - loss: 2.2255
Epoch 12/200
16/16 [==============================] - 1s 84ms/step - loss: 2.1938
Epoch 13/200
16/16 [=====

Ahora vamos a generar texto con el modelo de Casado

In [28]:
model = build_model_gru(vocab_size, embedding_dim, rnn_units, batch_size=1)
model.load_weights('./modelos/casado_weights.keras')

In [29]:
print(generate_text(model, start_string=u"La luz está muy cara"))

teareceones, electorezados.
Señores?
Señora. Leyes, ya enañesas.
Acabaremos, estemeza; estánes. ¿Parates.
Acabata, estempetencia.
Acabaremos?
Acabandonadas.
Señoras, aunquezarle. ¿Quezarecente, este, este, estemeación.
Señores? ¿Qué esabereza, estemplaza; electagaderales?
Señora? Señora?
Señores. Señor, Autónomentenezarte:?
¿Qué quedenea, estemezón. La Leye, ¿qué estemezó encepteraremoses, aquelezáles. Señores? ¿Que estemezarenes. Nosotros. Ya estabelezades. Leyes, ustedes. Ya electa: ¿persemos 


Podemos observar que aunque entrena muy rápido, el modelo de Pablo Casado no consigue formar palabras coherentes. Al cambiar la capa de LSTM a GRU el modelo no funciona bien, es posible que sea porque la GRU tiene menos parámetros y por lo tanto no es capaz de aprender bien el dataset.

### Conversación entre Pedro Sánchez y Santiago Abascal

Para similar una especie de conversación entre Pedro Sánchez y Santiago Abascal, vamos a darle como input a los modelos los outputs del otro modelo. Es decir, vamos a darle como input al modelo de Pedro Sánchez el texto generado por el modelo de Santiago Abascal, y viceversa.

In [31]:
import random
import tensorflow as tf

modelo_abascal = tf.keras.models.load_model('./modelos/abascal.keras')
modelo_sanchez = tf.keras.models.load_model('./modelos/sanchez.keras')

modelos = [modelo_abascal, modelo_sanchez]
politicos = ["Abascal", "Sanchez"]

ultima_palabra = u"España está en una situación"
turno = random.randint(0,1)

for i in range(5):
    turno = (turno + 1) % 2
    ultima_palabra = generate_text(modelos[turno], start_string=ultima_palabra)
    print(f"{politicos[turno]}: {ultima_palabra}\n")

    

Sanchez:  política. Sobre el plano del Gobierno, le diré que hemos tenido que gestionar más rápido, y espoco vamos avanzando, señor Casado, usted ya reconoce que no estamos en quiebra, que nos recuperamos. Ahí están las previsiones del Fondo Monetario Internacional: una caída bruselas, como he dicho antes, cuestionando la democracia. Y cuando la negociación entró en una fase de de ser ministro de Sanidad o que sea candidato a la Presidencia de la Generalitat de Catalunya, el señor Torra, es público y not

Abascal: ros hoy hemos dejado aquí el antídoto. De usted depende, la pelota está en su tejado. Muchas gracias. .
El balance de su gestión en dos años está siendo demolernación con la des aguantando sus insultos, dos años y cinco campañas. Usted ha cometido el error de venir aquí con un centenar de focos del virus que ha causado la muerte a 40 000 compatriotas. Ante su inacción fal su votación? Y, la tercera, ¿por qué bloquea el plan B jurídico para luchar contra la pandemia, como han 